In [0]:
!nvidia-smi

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip uninstall -y tensorflow
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
#!pip uninstall -y torch
#!pip install torch==1.4.0

In [0]:
import copy
import numpy as np
import pandas as pd

from tqdm.autonotebook import tqdm

from scipy.stats import norm
from scipy import stats
from sklearn.model_selection import KFold, StratifiedKFold

import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.utils.data

from transformers import *
from transformers import BertTokenizer
import tokenizers

import os
import re
import math
import random
from matplotlib import pyplot as plt
from math import floor, ceil
from sklearn.metrics import confusion_matrix
import seaborn as sns

plt.style.use("fivethirtyeight")
sns.set()
device = torch.device('cuda')
#torch.autograd.set_detect_anomaly(True)

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def get_best_start_end_idxs(_start_logits, _end_logits):
    best_logit = -1000
    best_idxs = None
    for start_idx, start_logit in enumerate(_start_logits):
        for end_idx, end_logit in enumerate(_end_logits[start_idx:]):
            logit_sum = (start_logit + end_logit).item()
            if logit_sum > best_logit:
                best_logit = logit_sum
                best_idxs = (start_idx, start_idx+end_idx)
    return best_idxs

def calculate_jaccard_score(
    original_tweet, 
    target_string, 
    sentiment_val, 
    idx_start, 
    idx_end, 
    offsets,
    verbose=False):
    
    if idx_end < idx_start:
        idx_end = idx_start
    
    filtered_output  = ""
    for ix in range(idx_start, idx_end + 1):
        filtered_output += original_tweet[offsets[ix][0]: offsets[ix][1]]
        if (ix+1) < len(offsets) and offsets[ix][1] < offsets[ix+1][0]:
            filtered_output += " "

    if sentiment_val == "neutral" or len(original_tweet.split()) < 2:
        filtered_output = original_tweet


    jac = jaccard(target_string.strip(), filtered_output.strip())
    return jac, filtered_output


class config:
    SEED = 43875210
    MAX_LEN = 128
    TRAIN_BATCH_SIZE = 16
    VALID_BATCH_SIZE = 8
    LR = 3e-5
    N_FOLDS = 5
    EPOCHS = 3
    PRETRAINED_WEIGHTS = "roberta-base"
    DRIVE_ROOT = "./drive/My Drive/Study/Tweet"
    #TRAIN_PATH = f"{DRIVE_ROOT}/input/train.csv"
    TRAIN_PATH = f"{DRIVE_ROOT}/input/all_tweet_data.csv"
    #MODEL_PATH = f"{DRIVE_ROOT}/model/org_sentiment_train"
    MODEL_PATH = f"{DRIVE_ROOT}/model/roberta_base"
    #TOKENIZER = tokenizers.ByteLevelBPETokenizer(vocab_file=f"{DRIVE_ROOT}/model/lm_train/vocab.json", merges_file=f"{DRIVE_ROOT}/model/lm_train/merges.txt", lowercase=False, add_prefix_space=True)
    TOKENIZER = tokenizers.ByteLevelBPETokenizer(vocab_file=f"{MODEL_PATH}/vocab.json", merges_file=f"{MODEL_PATH}/merges.txt", lowercase=True, add_prefix_space=True)
    UUID = "RoBERTa_base_20"
    OUT_DIR = f"{DRIVE_ROOT}/output/{UUID}/"

!mkdir -p "{config.OUT_DIR}"

sentiment_id = {s: config.TOKENIZER.encode(s).ids[0] for s in ["positive", "negative", "neutral"]}
#config.TOKENIZER.add_tokens(["[positive]", "[negative]", "[neutral]"])
#sentiment_id = {s: config.TOKENIZER.encode(f"[{s}]").ids[0] for s in ["positive", "negative", "neutral"]}
print(sentiment_id)

#import json
#with open(f"{config.MODEL_PATH}/vocab.json", "r") as f:
#    d = json.load(f)
#d["<mask>"]
MASK_ID = 50264

#train = pd.read_csv(config.TRAIN_PATH).dropna().reset_index(drop=True)
all_tweet_data = pd.read_csv(config.TRAIN_PATH)
train = all_tweet_data[all_tweet_data["data_type"] == "train"].reset_index(drop=True)
print("origin", len(train))
train = train[train["is_keep"]].reset_index(drop=True)
print("del duplicates", len(train))
train = train[["text", "selected_text", "new_sentiment", "sentiment"]]
train.columns = ["text", "selected_text", "sentiment", "org_sentiment"]

test = all_tweet_data[all_tweet_data["data_type"] != "train"].reset_index(drop=True)
print(test.shape[0])
test = test[test["is_keep"]].reset_index(drop=True)
test = test[test["text"].map(lambda x: len(x)) > 3].reset_index(drop=True)
test = test[test["text"].map(lambda x: len(set([xx for xx in x if xx not in [" ", ".", "!", "?", "_"]])) > 2 )].reset_index(drop=True)
print(test.shape[0])
test = test[["text", "text", "assumed_sentiment", "sentiment"]]
test.columns = ["text", "selected_text", "sentiment", "org_sentiment"]

#sentiment_label = {v: [i] for i, v in enumerate(train["org_sentiment"].unique())}
sentiment_label = {v: [config.TOKENIZER.encode(v).ids[0]] for v in train["org_sentiment"].unique()}
sentiment_label

In [0]:
def process_data(tweet, selected_text, sentiment, tokenizer, max_len, org_sentiment):
    tweet = " " + " ".join(str(tweet).split())
    selected_text = " " + " ".join(str(selected_text).split())

    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None

    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind+len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break

    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1

    tok_tweet = tokenizer.encode(tweet)
    input_ids_orig = tok_tweet.ids
    tweet_offsets = tok_tweet.offsets
    
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)

    try:
        targets_start = target_idx[0]
        targets_end = target_idx[-1]
    except IndexError:
        targets_start = 0
        targets_end = len(tweet) - 1     


    input_ids = [0] + [sentiment_id[sentiment]] + [2] + [2] + input_ids_orig + [2]
    token_type_ids = [0, 0, 0, 0] + [0] * (len(input_ids_orig) + 1)
    mask = [1] * len(token_type_ids)
    tweet_offsets = [(0, 0)] * 4 + tweet_offsets + [(0, 0)]
    targets_start += 4
    targets_end += 4

    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([1] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        tweet_offsets = tweet_offsets + ([(0, 0)] * padding_length)
    
    return {
        'ids': input_ids,
        'mask': mask,
        'token_type_ids': token_type_ids,
        'targets_start': targets_start,
        'targets_end': targets_end,
        'orig_tweet': tweet,
        'orig_selected': selected_text,
        'sentiment': sentiment,
        'offsets': tweet_offsets,
        'sentiment_label': sentiment_label[org_sentiment]
    }

class TweetDataset:
    def __init__(self, tweet, sentiment, selected_text, org_sentiment):
        self.tweet = tweet
        self.sentiment = sentiment
        self.selected_text = selected_text
        self.tokenizer = config.TOKENIZER
        self.max_len = config.MAX_LEN
        self.org_sentiment = org_sentiment
    
    def __len__(self):
        return len(self.tweet)

    def __getitem__(self, item):
        data = process_data(
            self.tweet[item], 
            self.selected_text[item],
            self.sentiment[item],
            self.tokenizer,
            self.max_len,
            self.org_sentiment[item]
        )
        
        max_idx = data["mask"].index(0)
        masked_idx = random.sample(list(range(4, max_idx)), 2)
        #masked_idx = [data["targets_start"], data["targets_end"]]
        

        dummy_ids = [MASK_ID if i in masked_idx else v for i, v in enumerate(data["ids"])]
        dummy_target = [v if i in masked_idx else -1 for i, v in enumerate(data["ids"])]
        
        dummy_target[1] = data["sentiment_label"][0]

        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
            'token_type_ids': torch.tensor(data["token_type_ids"], dtype=torch.long),
            'targets_start': torch.tensor(data["targets_start"], dtype=torch.long),
            'targets_end': torch.tensor(data["targets_end"], dtype=torch.long),
            'orig_tweet': data["orig_tweet"],
            'orig_selected': data["orig_selected"],
            'sentiment': data["sentiment"],
            'offsets': torch.tensor(data["offsets"], dtype=torch.long),
            'sentiment_label': torch.tensor(data["sentiment_label"], dtype=torch.long),
            'dummy_ids': torch.tensor(dummy_ids, dtype=torch.long),
            'dummy_target': torch.tensor(dummy_target, dtype=torch.long),
        }

In [0]:
def train_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train()
    losses, lrs = [], []
    tk0 = tqdm(data_loader, total=len(data_loader), desc="Training")
    for bi, d in enumerate(tk0):

        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        sentiment = d["sentiment"]
        orig_selected = d["orig_selected"]
        orig_tweet = d["orig_tweet"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        offsets = d["offsets"]
        sentiment_label = d["sentiment_label"]
        dummy_ids = d["dummy_ids"]
        dummy_target = d["dummy_target"]

        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets_start = targets_start.to(device, dtype=torch.long)
        targets_end = targets_end.to(device, dtype=torch.long)
        sentiment_label = sentiment_label.to(device, dtype=torch.long)
        dummy_ids = dummy_ids.to(device, dtype=torch.long)
        dummy_target = dummy_target.to(device, dtype=torch.long)

        model.zero_grad()
        #outputs_start, outputs_end, sentiment, lm = model(
        #outputs_start, outputs_end, lm = model(
        outputs_start, outputs_end = model(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids,
            is_train = True,
            dummy_ids=dummy_ids,
        )
        #loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end, hyp, ids)
        #loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end, hyp, sentiment_label)
        #loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end, sentiment, sentiment_label, lm, dummy_target)
        #loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end, lm, dummy_target)
        loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
        loss.backward()
        optimizer.step()
        scheduler.step()

        tk0.set_postfix(loss=loss.item())

        losses.append(float(loss))
        lrs.append(np.array([param_group["lr"] for param_group in optimizer.param_groups]).mean())

    return losses, lrs


def eval_fn(data_loader, model, device):
    model.eval()
    losses = []
    jaccards = []
    
    with torch.no_grad():
        tk0 = tqdm(data_loader, total=len(data_loader))
        for bi, d in enumerate(tk0):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            sentiment = d["sentiment"]
            orig_selected = d["orig_selected"]
            orig_tweet = d["orig_tweet"]
            targets_start = d["targets_start"]
            targets_end = d["targets_end"]
            offsets = d["offsets"].numpy()
            sentiment_label = d["sentiment_label"]

            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets_start = targets_start.to(device, dtype=torch.long)
            targets_end = targets_end.to(device, dtype=torch.long)
            sentiment_label = sentiment_label.to(device, dtype=torch.long)

            outputs_start, outputs_end = model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids
            )
            #loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end, hyp, ids)
            #loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end, hyp, sentiment_label)
            #loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
            outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
            outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
            jaccard_scores = []
            for px, tweet in enumerate(orig_tweet):
                selected_tweet = orig_selected[px]
                tweet_sentiment = sentiment[px]
                idx_start, idx_end = get_best_start_end_idxs(outputs_start[px, :], outputs_end[px, :])
                jaccard_score, _ = calculate_jaccard_score(
                    original_tweet=tweet,
                    target_string=selected_tweet,
                    sentiment_val=tweet_sentiment,
                    idx_start=idx_start,
                    idx_end=idx_end,
                    offsets=offsets[px]
                )
                jaccard_scores.append(jaccard_score)

            jaccards.append(np.mean(jaccard_scores))
            #losses.append(loss.item())
            losses.append(0.0)
    
    return np.mean(losses), np.mean(jaccards)

In [0]:
def pos_weight(pred_tensor, pos_tensor, neg_weight=1, pos_weight=1):
    gap = torch.argmax(pred_tensor, dim=1) - pos_tensor
    gap = gap.type(torch.float32)
    return torch.where(gap < 0, -neg_weight * gap, pos_weight * gap)


def KSLoss(preds, target):
    pred_cdf = torch.cumsum(torch.softmax(preds, dim=1), dim=1)
    target_cdf = torch.cumsum(target, dim=1)
    error = (target_cdf - pred_cdf)**2
    return torch.mean(error)

#def loss_fn(start_logits, end_logits, start_positions, end_positions, lm, ids):
#def loss_fn(start_logits, end_logits, start_positions, end_positions, h, t):
def loss_fn(start_logits, end_logits, start_positions, end_positions):
    loss_fct = nn.CrossEntropyLoss()
    #loss_fct = KSLoss
    start_loss = loss_fct(start_logits, start_positions)
    end_loss = loss_fct(end_logits, end_positions)

    #loss_fct = nn.CrossEntropyLoss(reduction="none")
    #start_loss = loss_fct(start_logits, start_positions) * pos_weight(start_logits, start_positions, 1, 1)
    #end_loss = loss_fct(end_logits, end_positions) * pos_weight(end_logits, end_positions, 1, 1)
    #start_loss = torch.mean(start_loss)
    #end_loss = torch.mean(end_loss)

    #sentiment_loss = nn.CrossEntropyLoss()(sentiment, sentiment_label[:, 0])
    #t = t[:, 1:].contiguous()
    #lm_loss = nn.CrossEntropyLoss(ignore_index=-1)(lm, ids.view(-1))
    
    # 3:3:2:2
    total_loss = start_loss + end_loss #+ lm_loss #+ sentiment_loss
    return total_loss
    
class RoBERTaForTweet(nn.Module):
    def __init__(self):
        super(RoBERTaForTweet, self).__init__()

        conf = RobertaConfig.from_pretrained(config.MODEL_PATH)
        conf.output_hidden_states = True

        self.n_use_layer = 2
        self.n_vocab = conf.vocab_size

        n_weights = conf.num_hidden_layers + 1
        weights_init = torch.zeros(n_weights).float()
        weights_init.data[:-1] = -3
        self.layer_weights = torch.nn.Parameter(weights_init)

        self.roberta = RobertaModel.from_pretrained(config.MODEL_PATH, config=conf)
        self.dropout = nn.Dropout(0.2)

        #self.dropouts = nn.ModuleList([nn.Dropout(0.5) for _ in range(5)])
        #self.qa_outputs = nn.Linear(768, 2)


        self.conv = nn.Conv1d(in_channels=768, out_channels=2, kernel_size=1)
        #self.fc = nn.Linear(128, 128)

        #self.sentiment_dropout = nn.Dropout(0.2)
        #self.sentiment_outputs = nn.Linear(768, 13)
        #self.sentiment_outputs = nn.Linear(768*self.n_use_layer, self.n_vocab)
        #self.sentiment_outputs = modeling_bert.BertOnlyMLMHead(conf)

        #self.lm_dropout = nn.Dropout(0.2)
        #self.lm_outputs = nn.Linear(768, self.n_vocab)
        #self.lm_outputs = modeling_bert.BertOnlyMLMHead(conf)
        self.lm_outputs = modeling_roberta.RobertaLMHead(conf)
        #self.cls = modeling_bert.BertOnlyMLMHead(conf)

    def forward(self, ids, mask, token_type_ids, is_train=False, dummy_ids=None):
        _, _, out = self.roberta(ids, attention_mask=mask, token_type_ids=token_type_ids)

        #sequence_output = torch.cat([out[(i+1)*-1] for i in range(self.n_use_layer)], dim=2)
        #sequence_output = self.dropout(sequence_output)
        #logits = self.qa_outputs(sequence_output)

        sequence_output = torch.stack([self.dropout(layer) for layer in out], dim=3)
        sequence_output = (torch.softmax(self.layer_weights, dim=0) * sequence_output).sum(-1)
        #logits = sum([self.qa_outputs(dropout(sequence_output)) for dropout in self.dropouts])/5
        
        h = self.conv(sequence_output.transpose(1, 2))
        #h = F.relu(h)
        #logits = self.fc(h).transpose(1, 2)
        logits = h.transpose(1, 2)

        start_logits, end_logits = logits.split(1, dim=-1)

        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)


        return start_logits, end_logits

        if is_train:
            #cat_output = torch.cat([out[(i+1)*-1][:, 1] for i in range(1)], dim=1)
            #sentiment = self.sentiment_outputs(cat_output)
            #sentiment = self.sentiment_outputs(out[-1])
            #sentiment = sentiment[:, 1]

            lm_outputs, _, _ = self.roberta(dummy_ids, attention_mask=mask, token_type_ids=token_type_ids)
            #lm_outputs = self.lm_dropout(out[-1])
            lm = self.lm_outputs(lm_outputs)
            #h = h[:, :-1, :].contiguous()
            #o1, _, _ = self.roberta(dummy_ids, attention_mask=mask, token_type_ids=token_type_ids)
            #h = self.cls(o1)
            lm = lm.view(-1, self.n_vocab)

            return start_logits, end_logits, lm

        return start_logits, end_logits

In [0]:
#x = torch.ones([32, 128, 768, 13])
x = torch.ones([32, 128, 768])
x.shape

In [0]:
conv = nn.Conv1d(in_channels=768, out_channels=2, kernel_size=1)
fc = nn.Linear(128, 128)

In [0]:
32, 128, 1

In [0]:
h = conv(x.transpose(1, 2)) #
h.shape

In [0]:
logit = fc(h).transpose(1, 2)
s, e = logit.split(1, dim=-1)

In [0]:
s.shape , e.shape

In [0]:
def pseudo_fn(data_loader, model, device):
    final_output = []
    model.eval()
    with torch.no_grad():
        tk0 = tqdm(data_loader, total=len(data_loader))
        for bi, d in enumerate(tk0):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            sentiment = d["sentiment"]
            orig_selected = d["orig_selected"]
            orig_tweet = d["orig_tweet"]
            targets_start = d["targets_start"]
            targets_end = d["targets_end"]
            offsets = d["offsets"].numpy()
            sentiment_label = d["sentiment_label"]

            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets_start = targets_start.to(device, dtype=torch.long)
            targets_end = targets_end.to(device, dtype=torch.long)
            sentiment_label = sentiment_label.to(device, dtype=torch.long)

            outputs_start, outputs_end = model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids
            )

            outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
            outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
            jaccard_scores = []
            for px, tweet in enumerate(orig_tweet):
                selected_tweet = orig_selected[px]
                tweet_sentiment = sentiment[px]
                idx_start, idx_end = get_best_start_end_idxs(outputs_start[px, :], outputs_end[px, :])
                _, output_sentence = calculate_jaccard_score(
                    original_tweet=tweet,
                    target_string=selected_tweet,
                    sentiment_val=tweet_sentiment,
                    idx_start=idx_start,
                    idx_end=idx_end,
                    offsets=offsets[px]
                )
                final_output.append(output_sentence)
    
    return final_output

In [0]:
def get_pseudo_label(df_test, df_train, model, device):
    test_dataset = TweetDataset(
        tweet=df_test.text.values,
        sentiment=df_test.sentiment.values,
        selected_text=df_test.selected_text.values,
        org_sentiment=df_test.org_sentiment.values
    )

    test_data_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=config.TRAIN_BATCH_SIZE,
        num_workers=0,
        shuffle=False
    )
    final_output = pseudo_fn(test_data_loader, model, device)

    df_test["selected_text"] = [j if len(j.split()) == 1 else i for i, j in zip(df_test["text"], final_output)]
 
    df_pseudo = pd.concat([df_train, df_test], axis=0).reset_index(drop=True)

    train_dataset = TweetDataset(
        tweet=df_pseudo.text.values,
        sentiment=df_pseudo.sentiment.values,
        selected_text=df_pseudo.selected_text.values,
        org_sentiment=df_pseudo.org_sentiment.values
    )

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.TRAIN_BATCH_SIZE,
        num_workers=0,
        shuffle=True
    )

    return train_data_loader

In [0]:
skf = StratifiedKFold(n_splits=config.N_FOLDS, shuffle=True, random_state=config.SEED).split(train["sentiment"].values, train["sentiment"].values)

train_log_dfs, valid_log_dfs = [], []
for fold, (train_idx, valid_idx) in enumerate(skf):
    print(f"### Fold-{fold} ###")
    #if fold in [0, 1, 2]:
    #    continue
    seed_everything(config.SEED + fold)

    df_train = train.iloc[train_idx].reset_index(drop=True)
    df_valid = train.iloc[valid_idx].reset_index(drop=True)

    train_dataset = TweetDataset(
        tweet=df_train.text.values,
        sentiment=df_train.sentiment.values,
        selected_text=df_train.selected_text.values,
        org_sentiment=df_train.org_sentiment.values
    )

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.TRAIN_BATCH_SIZE,
        num_workers=0,
        shuffle=True
    )

    valid_dataset = TweetDataset(
        tweet=df_valid.text.values,
        sentiment=df_valid.sentiment.values,
        selected_text=df_valid.selected_text.values,
        org_sentiment=df_valid.org_sentiment.values
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=config.VALID_BATCH_SIZE,
        num_workers=0,
        shuffle=False
    )

    model = RoBERTaForTweet()
    model.to(device)

    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    ]
    optimizer = AdamW(optimizer_parameters, lr=config.LR)
    
    num_train_steps = int(len(df_train) / config.TRAIN_BATCH_SIZE * config.EPOCHS)
    #scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)
    
    t_res, v_res = [], []
    best_score = 0
    for epoch in range(config.EPOCHS):
        train_loss, lrs = train_fn(train_data_loader, model, optimizer, device, scheduler=scheduler)
        valid_loss, valid_jaccard = eval_fn(valid_data_loader, model, device)
        print(f"{epoch} epoch, jaccard={valid_jaccard}")
        if best_score < valid_jaccard:
            print(f"  --> Best Model Update!!")
            best_score = valid_jaccard
            torch.save(model.state_dict(), f"{config.OUT_DIR}/roberta_f{fold}_best.bin")
        t_res.append(pd.DataFrame(zip(train_loss, lrs), columns=["train_loss", "lrs"]))
        v_res.append((valid_loss, valid_jaccard))

        #train_data_loader = get_pseudo_label(test, df_train, model, device)

    train_log_dfs.append(pd.concat(t_res, axis=0).reset_index(drop=True))
    valid_log_dfs.append(pd.DataFrame(v_res, columns=["valid_loss", "jaccard_score"]))

In [0]:
for fold, (t_log, v_log) in enumerate(zip(train_log_dfs, valid_log_dfs)):
    print(f"### {fold}-Fold ###")
    display(v_log)
    t_log["train_loss"].plot(title="train loss");
    plt.show()
    t_log["lrs"].plot(title="learning rate");
    plt.show()

In [0]:
!ls